In [1]:
library("discover")
library("tidyverse")

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1   dplyr_1.1.4    
 [5] purrr_1.0.2     readr_2.1.5     tidyr_1.3.0     tibble_3.2.1   
 [9] ggplot2_3.4.4   tidyverse_2.0.0 discover_0.9.4 

loaded via a namespace (and not attach

In [3]:
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

# TCGA

In [4]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [5]:
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. ..$ truncating: num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. .. ..$ mutation: num [1:9082] 56 47 40 422 214 588 14 8 19 8 ...
  .. ..$ truncating:'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-

In [7]:
globa_gam<-run_data$M$M[[1]]+run_data$M$M[[2]][rownames(run_data$M$M[[1]]),colnames(run_data$M$M[[1]])]
globa_gam[globa_gam>1]<-1

In [8]:
dim(globa_gam)

[1]  396 9082

In [ ]:
gam_discover <-discover.matrix(globa_gam[,names(run_data$sample.class)],strata = unlist(run_data$sample.class))

In [ ]:
result.mutex <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='less')
result.co <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='greater')
ME_discover <- as.data.frame(result.mutex,q.threshold=0.25)
CO_discover <- as.data.frame(result.co,q.threshold=0.25)
discover_result <- rbind(ME_discover,CO_discover)

In [ ]:
saveRDS(discover_result,file='../results/Discover/tcga_all.rds')

## TCGA sampling 

In [10]:
sampling_list <- readRDS('/mnt/ptemp/arvind/robustness_analysis/TCGA robustness analysis/data/sampling_list.rds')

In [11]:
sample_list<- sampling_list[26:30]

In [12]:
str(sample_list)

List of 5
 $ : Named chr [1:7266] "BRCA_LumA" "THYM" "BRCA_LumA" "LIHC" ...
  ..- attr(*, "names")= chr [1:7266] "TCGA-A2-A0YD-01" "TCGA-ZB-A96D-01" "TCGA-AO-A0JF-01" "TCGA-KR-A7K2-01" ...
 $ : Named chr [1:7266] "BLCA" "HNSC_HPV-" "OV" "CHOL" ...
  ..- attr(*, "names")= chr [1:7266] "TCGA-XF-A9SI-01" "TCGA-UF-A7JO-01" "TCGA-61-2110-01" "TCGA-W5-AA38-01" ...
 $ : Named chr [1:7266] "BLCA" "CRC_CIN" "MESO" "GBM" ...
  ..- attr(*, "names")= chr [1:7266] "TCGA-E5-A4U1-01" "TCGA-DM-A285-01" "TCGA-ZN-A9VS-01" "TCGA-06-2558-01" ...
 $ : Named chr [1:7266] "UCEC_MSI" "BRCA_LumA" "SARC_LMS" "BRCA_LumB" ...
  ..- attr(*, "names")= chr [1:7266] "TCGA-AX-A1C4-01" "TCGA-AQ-A1H2-01" "TCGA-DX-A6Z2-01" "TCGA-AR-A24Z-01" ...
 $ : Named chr [1:7266] "BRCA_Basal" "LAML" "SKCM" "BRCA_Her2" ...
  ..- attr(*, "names")= chr [1:7266] "TCGA-E9-A22G-01" "TCGA-AB-2815-03" "TCGA-GN-A265-06" "TCGA-A8-A09G-01" ...


In [13]:
dim(globa_gam)

[1]  396 9082

In [ ]:
results_discover <- list()
k=1
for (obj in sample_list[1:5] ){
    print(paste('#Running run-> ',k))
    samples<-names(obj)
    local_gam <- globa_gam[,samples]
    print(dim(local_gam))
    gam_discover <-discover.matrix(local_gam,strata = unlist(obj))
    
    # Generating Results discover
    result.mutex <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='less')
    result.co <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='greater')
    ME_discover <- as.data.frame(result.mutex,q.threshold=0.25)
    CO_discover <- as.data.frame(result.co,q.threshold=0.25)
    discover_result <- rbind(ME_discover,CO_discover)
    discover_result$type <- c(rep('ME',nrow(ME_discover)),rep('CO',nrow(CO_discover)))
    results_discover[[k]]<-discover_result
    k=k+1

}

In [ ]:
str(results_discover)

In [ ]:
i=1
for(obj in results_discover){
    saveRDS(obj,file=paste('/mnt/ptemp/arvind/tool_comaprision/results/Discover/sampling_run_tcga_result_80_',i,'.rds',sep=""))        
    i=i+1
}

In [ ]:
saveRDS(results_discover,file='/mnt/ptemp/arvind/tool_comaprision/results/Discover/sampling_run_brca_result_80_list.rds')

# LUAD

In [14]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [15]:
sample_covariates<-run_data$sample.class

In [16]:
luad_samples <- names(sample_covariates[(which(sample_covariates %in% c('LUAD')))])

In [17]:
data<- filter_run_data(run_data = run_data,gene_order = names(run_data$alteration.class),sample_order = luad_samples)

In [18]:
globa_gam<-data$M$M[[1]]+data$M$M[[2]][rownames(data$M$M[[1]]),colnames(data$M$M[[1]])]
globa_gam[globa_gam>1]<-1

In [19]:
dim(globa_gam)

[1] 396 502

In [ ]:
gam_discover <-discover.matrix(globa_gam[,names(run_data$sample.class)],strata = unlist(run_data$sample.class))

In [ ]:
result.mutex <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='less')
result.co <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='greater')
ME_discover <- as.data.frame(result.mutex,q.threshold=0.25)
CO_discover <- as.data.frame(result.co,q.threshold=0.25)
discover_result <- rbind(ME_discover,CO_discover)

In [ ]:
saveRDS(discover_result,file='../results/Discover/luad_all.rds')

## LUAD sampling 

In [20]:
sample_list1 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad.rds')
sample_list2 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad_2.rds')

In [21]:
str(c(sample_list1[11:15],sample_list2[11:15]))

List of 10
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-78-7539-01" "TCGA-99-8033-01" "TCGA-73-4670-01" "TCGA-75-5146-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-75-6214-01" "TCGA-64-5775-01" "TCGA-97-A4M3-01" "TCGA-93-A4JO-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-64-1679-01" "TCGA-64-1676-01" "TCGA-55-8505-01" "TCGA-97-8547-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-55-8204-01" "TCGA-55-8094-01" "TCGA-49-AARE-01" "TCGA-44-A47G-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-05-4422-01" "TCGA-49-AAR9-01" "TCGA-05-4417-01" "TCGA-55-6984-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-91-6848-01" "TCGA-55-6971-01" "TCGA-S2-AA1A-01" "TCGA-86-

In [ ]:
sample_list<-c(sample_list1[11:15],sample_list2[11:15])

In [ ]:
results_discover <- list()
k=1
for (obj in sample_list[11:20] ){
    print(paste('#Running run-> ',k))
    samples<-names(obj)
    local_gam <- globa_gam[,samples]
    print(dim(local_gam))
    gam_discover <-discover.matrix(local_gam)
    
    # Generating Results discover
    result.mutex <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='less')
    result.co <- pairwise.discover.test(gam_discover,fdr.method='DBH',alternative='greater')
    ME_discover <- as.data.frame(result.mutex,q.threshold=0.25)
    CO_discover <- as.data.frame(result.co,q.threshold=0.25)
    discover_result <- rbind(ME_discover,CO_discover)
    discover_result$type <- c(rep('ME',nrow(ME_discover)),rep('CO',nrow(CO_discover)))
    results_discover[[k]]<-discover_result
    k=k+1

}

In [ ]:
str(results_discover)

In [ ]:
i=1
for(obj in results_discover){
    saveRDS(obj,file=paste('/mnt/ptemp/arvind/tool_comaprision/results/Discover/sampling_run_luad_result_80_',i,'.rds',sep=""))        
    i=i+1
}

In [ ]:
saveRDS(results_discover,file='/mnt/ptemp/arvind/tool_comaprision/results/Discover/sampling_run_luad_result_80_list.rds')